In [4]:
import os
import re
import pandas as pd

# === SETTINGS ===
base_dir = r"C:\Users\polla\uk-kids-veggie-analysis"
data_file = os.path.join(base_dir, "data", "ndns-2019-to-2023_data_chapter-4.ods")
output_dir = os.path.join(base_dir, "data", "sheets")

# Create output folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# === FUNCTION TO CLEAN SHEET NAMES ===
def clean_name(name):
    name = name.lower().strip()
    name = re.sub(r"[^\w]+", "_", name)
    return name.strip("_")

# === LOAD AND EXPORT EACH SHEET ===
print("🔍 Reading all sheets from:", data_file)
sheets = pd.read_excel(data_file, sheet_name=None, engine="odf")

for sheet_name, df in sheets.items():
    cleaned_name = clean_name(f"ndns_2019_2023_{sheet_name}")
    output_path = os.path.join(output_dir, f"{cleaned_name}.csv")

    # Drop fully empty rows/columns
    df = df.dropna(how="all").dropna(axis=1, how="all")

    # Export as CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Saved {sheet_name} → {output_path}")

print("\n🎉 All sheets extracted and saved as CSV.")


🔍 Reading all sheets from: C:\Users\polla\uk-kids-veggie-analysis\data\ndns-2019-to-2023_data_chapter-4.ods
✅ Saved Cover_sheet → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_cover_sheet.csv
✅ Saved 4_1a → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_1a.csv
✅ Saved 4_1b → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_1b.csv
✅ Saved 4_1c → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_1c.csv
✅ Saved 4_2a → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_2a.csv
✅ Saved 4_2b → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_2b.csv
✅ Saved 4_2c → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_2c.csv
✅ Saved 4_3a → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_3a.csv
✅ Saved 4_3b → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_3b.csv
✅ Saved 4_4 → C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndn

In [11]:
import pandas as pd

# ✅ Correct file path
file_path = r"C:\Users\polla\uk-kids-veggie-analysis\data\sheets\ndns_2019_2023_4_1a.csv"

# Load WITHOUT header so we can see metadata and header rows
df_raw = pd.read_csv(file_path, header=None)

# 🔍 View top 20 rows to inspect where the real table begins
print("\n🔍 Top 20 rows:\n")
print(df_raw.head(20))

# 📊 See how many columns are populated in each row (to spot start of actual data)
print("\n📊 Non-empty column counts per row:\n")
print(df_raw.notna().sum(axis=1).head(100))





🔍 Top 20 rows:

                                                   0                       1   \
0   Table 4.1a: Total quantities of food consumed ...              Unnamed: 1   
1   This worksheet contains one table. Some cells ...                     NaN   
2   National Diet and Nutrition Survey. Results fr...                     NaN   
3                         Males aged 4 years and over                     NaN   
4                                        2019 to 2023                     NaN   
5   Blank cells occur in rows 8, 27, 40, 48, 62, 6...                     NaN   
6         Food Group[note 1][note 2][note 3][note 10]  Boys 4 to 10 yearsMean   
7                         Cereals and cereal products                     NaN   
8                                               Pizza        25.9617210668341   
9                                               Pasta        14.1070841954569   
10                                 Pasta-based dishes         11.197119098337   
11         

In [17]:
import pandas as pd
import re
import os
from pathlib import Path

# === Paths ===
input_dir = Path("C:/Users/polla/uk-kids-veggie-analysis/data/sheets")
output_dir = input_dir.parent / "cleaned"
output_dir.mkdir(exist_ok=True)

# === Helpers ===
def extract_title(file_path):
    """Extract A1 cell (first row, col 0) as the table title"""
    try:
        title = pd.read_csv(file_path, header=None, nrows=1).iloc[0, 0]
        return title if isinstance(title, str) else "Unknown title"
    except Exception:
        return "Unknown title"

def shorten_title(title):
    """Shorten title for filename use"""
    match = re.search(r'Table\s+([\d.]+[a-zA-Z]?)', title)
    table_id = match.group(1).replace('.', '_') if match else "unknown"

    title_lower = title.lower()
    if "male" in title_lower:
        desc = "male_food_quantities_by_age"
    elif "female" in title_lower:
        desc = "female_food_quantities_by_age"
    elif "nutrient" in title_lower:
        desc = "nutrient_intake"
    elif "energy" in title_lower:
        desc = "energy_intake"
    else:
        desc = "food_data"

    return f"ndns_2019_2023_{table_id}_{desc}_cleaned.csv"

def clean_notes(text):
    """Remove [note x] references from column names and values"""
    if isinstance(text, str):
        return re.sub(r'\s*\[note.*?\]', '', text).strip()
    return text

def find_header_row(df):
    """Find the likely header row (at least 5 non-empty cells)"""
    for i, row in df.iterrows():
        if row.count() >= 5:
            return i
    return 6  # default fallback

# === Process All Files ===
for file in input_dir.glob("ndns_2019_2023_4_*.csv"):
    print(f"📂 Processing: {file.name}")

    try:
        # === Step 1: Get title from A1 ===
        title = extract_title(file)
        cleaned_name = shorten_title(title)
        output_path = output_dir / cleaned_name

        # === Step 2: Load raw content ===
        df_raw = pd.read_csv(file, header=None)
        header_row = find_header_row(df_raw)

        # === Step 3: Extract and clean data ===
        df = df_raw.iloc[header_row + 1:].reset_index(drop=True)
        df.columns = df_raw.iloc[header_row]
        df = df.dropna(how="all")
        df = df.loc[:, ~df.columns.isnull()]
        df.columns = [clean_notes(col) for col in df.columns]
        df.iloc[:, 0] = df.iloc[:, 0].apply(clean_notes)

        # Drop metadata or footnote rows
        df = df[~df[df.columns[0]].astype(str).str.lower().str.startswith("note")]
        df = df[~df[df.columns[0]].astype(str).str.lower().str.startswith("table")]

        # === Step 4: Save ===
        df.to_csv(output_path, index=False)
        print(f"✅ Saved: {output_path.name}")
    
    except Exception as e:
        print(f"❌ Error processing {file.name}: {e}")


📂 Processing: ndns_2019_2023_4_10.csv
✅ Saved: ndns_2019_2023_4_10_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_11.csv
✅ Saved: ndns_2019_2023_4_11_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_12.csv
✅ Saved: ndns_2019_2023_4_12_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_13.csv
✅ Saved: ndns_2019_2023_4_13_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_14.csv
✅ Saved: ndns_2019_2023_4_14_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_15.csv
✅ Saved: ndns_2019_2023_4_15_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_16.csv
✅ Saved: ndns_2019_2023_4_16_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_17.csv
✅ Saved: ndns_2019_2023_4_17_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_18.csv
✅ Saved: ndns_2019_2023_4_18_food_data_cleaned.csv
📂 Processing: ndns_2019_2023_4_1a.csv
✅ Saved: ndns_2019_2023_4_1a_male_food_quantities_by_age_cleaned.csv
📂 Processing: ndns_2019_2023_4_1b.csv
✅ Saved: ndns_2019_2023_4_1b_male_food_quantities_by_a